In [1]:
import cv2
import os
import time

def extract_frames(video_path, output_dir="frames"):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)
    print(f"🎞️ Extracting frames from '{video_path}'...")
    while True:
        ret, frame = cap.read()
        if not ret: break
        cv2.imwrite(f"{output_dir}/frame_{count:06d}.jpg", frame)
        count += 1
    cap.release()
    print(f"✅ Extracted {count} frames at {fps} FPS")
    return count, fps

if __name__ == "__main__":
    start = time.time()
    total, fps = extract_frames("jumbled_video.mp4")
    print(f"⏱️ Done in {time.time()-start:.2f}s")

🎞️ Extracting frames from 'jumbled_video.mp4'...
✅ Extracted 0 frames at 30 FPS
⏱️ Done in 0.09s


In [4]:
# =====================================
# DAY 2: GPU Detection + SSIM Kernel
# =====================================
import torch
import torch.nn.functional as F

# --------------------------
# 🧠 1. Detect and Set Device
# --------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("🚀 Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("⚙️ Using CPU (no GPU found)")

# -----------------------------------------
# 🧩 2. Create Gaussian Kernel for SSIM
# -----------------------------------------
def gaussian_kernel(size=11, sigma=1.5, channels=3):
    """Creates a 2D Gaussian kernel expanded across channels."""
    coords = torch.arange(size, dtype=torch.float32) - size // 2
    grid = coords ** 2
    kernel_1d = torch.exp(-grid / (2 * sigma ** 2))
    kernel_1d /= kernel_1d.sum()
    kernel_2d = kernel_1d[:, None] @ kernel_1d[None, :]
    kernel_2d = kernel_2d.expand(channels, 1, size, size).to(device)
    return kernel_2d

# -----------------------------------------
# 🧮 3. SSIM Function (basic building block)
# -----------------------------------------
def ssim_torch(img1, img2, kernel, C1=0.01**2, C2=0.03**2):
    """Compute SSIM between two images using Gaussian kernel (Torch)."""
    mu1 = F.conv2d(img1, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1])
    mu2 = F.conv2d(img2, kernel, padding=kernel.shape[-1] // 2, groups=img2.shape[1])

    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1]) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, kernel, padding=kernel.shape[-1] // 2, groups=img2.shape[1]) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1]) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    )
    return ssim_map.mean()

# -----------------------------------------
# ✅ 4. Example Test (optional)
# -----------------------------------------
if __name__ == "__main__":
    # Create two dummy images (360x640 RGB)
    img1 = torch.rand(1, 3, 360, 640).to(device)
    img2 = torch.rand(1, 3, 360, 640).to(device)
    kernel = gaussian_kernel()
    val = ssim_torch(img1, img2, kernel)
    print(f"🔍 Test SSIM Value: {val.item():.4f}")


⚙️ Using CPU (no GPU found)
🔍 Test SSIM Value: 0.0162
